 <span>

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

</span>


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

- Interceptor: txt database
- we take till rank 200 and not first 200 rows
- find vs findall
- NYU issue
- Indian Institute of Science (IISc) Bangalore issue
- json['data'][0].keys()

In [17]:
def find_div_and_number(soup, class_name):
    
    div = soup.find('div', class_= class_name)
    if div is not None:
        return div.find('div', class_='number').text.replace('\n', '').replace(',', '')
    else: return np.NaN
        

def parse_uni_details(url):
    page_body = requests.get('https://topuniversities.com' + uni['url']).text

    soup = BeautifulSoup(page_body, "html.parser")
    
    faculty_total = find_div_and_number(soup, 'total faculty')
    faculty_intl = faculty_total = find_div_and_number(soup, 'inter faculty')
    student_total = faculty_total = find_div_and_number(soup, 'total student')
    student_intl = faculty_total = find_div_and_number(soup, 'total inter')
    
    return faculty_total, faculty_intl, student_total, student_intl

In [18]:
#ex1.1 
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508153509501')
json = r.json()

topuniversities = pd.DataFrame()

for (idx, uni) in enumerate(json['data']):
    
    if int(uni['rank_display'].replace('=','')) > 200:
        break
        
    faculty_total, faculty_intl, student_total, student_intl = parse_uni_details(uni['url'])
        
    entry = pd.DataFrame({'name': uni['title'], 'rank': uni['rank_display'].replace('=', ''), 'country': uni['country'],
                          'region': uni['region'], 'faculty_intl': faculty_intl, 'faculty_total': faculty_total,
                          'student_intl': student_intl, 'student_total': student_total}, index=[idx]) 
    topuniversities = topuniversities.append(entry)

# Rearrange columns order
topuniversities = topuniversities[['name', 'rank', 'country', 'region', 'faculty_intl', 'faculty_total', 'student_intl', 'student_total']]

# Change rank, faculty and student cols to numeric value
topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']]\
    = topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']].apply(pd.to_numeric)
    
# Serialization as a pickle file
topuniversities.to_pickle('pickled/topuniversities.pkl')
    
topuniversities.head()

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,3717.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,3611.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,5266.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,647.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,6699.0,6699.0,18770.0


In [19]:
topuniversities = pd.read_pickle('pickled/topuniversities.pkl')
topuniversities.head()

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,3717.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,3611.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,5266.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,647.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,6699.0,6699.0,18770.0


In [20]:
def getRatioSF (source ) :
    res = topuniversities[['name']]
    res['ratio faculty/student'] = topuniversities['faculty_total']/topuniversities['student_total']
    return res.sort_values('ratio faculty/student',ascending=False).dropna()

In [21]:
#1.2 a
dataex1a = getRatioSF(topuniversities)
dataex1a

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,ratio faculty/student
34,London School of Economics and Political Scien...,0.691393
11,Ecole Polytechnique Fédérale de Lausanne (EPFL),0.570047
7,Imperial College London,0.543567
198,Maastricht University,0.502533
47,Carnegie Mellon University,0.478062
6,UCL (University College London),0.477928
91,University of St Andrews,0.457955
41,The University of Melbourne,0.427434
126,Queen Mary University of London,0.421816
25,The University of Hong Kong,0.407144


In [31]:
def getRatioInter (source ) :
    res = topuniversities[['name']]
    res['ratio international student'] = topuniversities['student_intl']/topuniversities['student_total']
    return res.sort_values('ratio international student',ascending=False).dropna()

In [32]:
#1.2 b
dataex1b = getRatioInter(topuniversities)
dataex1b

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,ratio international student
34,London School of Economics and Political Scien...,0.691393
11,Ecole Polytechnique Fédérale de Lausanne (EPFL),0.570047
7,Imperial College London,0.543567
198,Maastricht University,0.502533
47,Carnegie Mellon University,0.478062
6,UCL (University College London),0.477928
91,University of St Andrews,0.457955
41,The University of Melbourne,0.427434
126,Queen Mary University of London,0.421816
25,The University of Hong Kong,0.407144


In [33]:
def getRatioSFBy (source, by ) :
    res = topuniversities.groupby(by,as_index=False).sum()
    res['ratio faculty/student'] = res['faculty_total']/res['student_total']
    return res[[by,'ratio faculty/student']].sort_values('ratio faculty/student',ascending=False).dropna()

In [34]:
def getRatioInterBy (source, by ) :
    res = topuniversities.groupby(by,as_index=False).sum()
    res['ratio international student'] = res['student_intl']/res['student_total']
    return res[[by,'ratio international student']].sort_values('ratio international student',ascending=False).dropna()

In [35]:
#2.2 c ratio between faculty members and students
dataex2cSF = getRatioSFBy(topuniversities,'country')
dataex2cSF

,country,ratio faculty/student
1,Australia,0.352189
32,United Kingdom,0.341705
12,Hong Kong,0.310751
2,Austria,0.309980
30,Switzerland,0.302396
25,Singapore,0.276537
5,Canada,0.260161
21,New Zealand,0.258215
14,Ireland,0.235299
20,Netherlands,0.232980


In [36]:
#2.2 c ratio of international students
dataex2cInter = getRatioInterBy(topuniversities,'country')
dataex2cInter

,country,ratio international student
1,Australia,0.352189
32,United Kingdom,0.341705
12,Hong Kong,0.310751
2,Austria,0.309980
30,Switzerland,0.302396
25,Singapore,0.276537
5,Canada,0.260161
21,New Zealand,0.258215
14,Ireland,0.235299
20,Netherlands,0.232980


In [37]:
#2.2 c ratio between faculty members and students
dataex2dSF = getRatioSFBy(topuniversities,'region')
dataex2dSF

,region,ratio faculty/student
5,Oceania,0.339261
2,Europe,0.229589
4,North America,0.188906
0,Africa,0.169703
1,Asia,0.136431
3,Latin America,0.087520


In [38]:
#2.2 d ratio of international students
dataex2dInter = getRatioInterBy(topuniversities,'region')
dataex2dInter

,region,ratio international student
5,Oceania,0.339261
2,Europe,0.229589
4,North America,0.188906
0,Africa,0.169703
1,Asia,0.136431
3,Latin America,0.087520


In [39]:
#ex2.2
r = requests.get('https://www.timeshighereducation.com/world-university-rankings/2017/world-ranking')
print(r.text)
json = r.json()

topuniversities = pd.DataFrame()

for (idx, uni) in enumerate(json['data']):
    
    if int(uni['rank_display'].replace('=','')) > 200:
        break
        
    faculty_total, faculty_intl, student_total, student_intl = parse_uni_details(uni['url'])
        
    entry = pd.DataFrame({'name': uni['title'], 'rank': uni['rank_display'].replace('=', ''), 'country': uni['country'],
                          'region': uni['region'], 'faculty_intl': faculty_intl, 'faculty_total': faculty_total,
                          'student_intl': student_intl, 'student_total': student_total}, index=[idx]) 
    topuniversities = topuniversities.append(entry)

# Rearrange columns order
topuniversities = topuniversities[['name', 'rank', 'country', 'region', 'faculty_intl', 'faculty_total', 'student_intl', 'student_total']]

# Change rank, faculty and student cols to numeric value
topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']]\
    = topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']].apply(pd.to_numeric)
    
# Serialization as a pickle file
topuniversities.to_pickle('pickled/topuniversities.pkl')
    
topuniversities.head()

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"
  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html lang="en" dir="ltr"
  xmlns:og="http://ogp.me/ns#"
  xmlns:article="http://ogp.me/ns/article#"
  xmlns:book="http://ogp.me/ns/book#"
  xmlns:profile="http://ogp.me/ns/profile#"
  xmlns:video="http://ogp.me/ns/video#"
  xmlns:product="http://ogp.me/ns/product#">
<head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<link rel="shortcut icon" href="https://www.timeshighereducation.com/sites/default/themes/custom/the_responsive/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="apple-itunes-app" content="app-id=578411515" />
<meta name="twitter:site" content="@timeshighered" />
<meta name="twitter:card" content="summary_large_image" />
<meta name="twitter:title" content="World University Rankings 2016-2017

JSONDecodeError: Expecting value: line 1 column 1 (char 0)